In [8]:
import os
import string
import re

In [9]:
#Checks for which files are available for use
path = "raw_XML/Everything"
listOfFiles = os.listdir(path)
print(listOfFiles)

['Anabasis.perseus-grc2.xml', 'Apology.perseus-grc2.xml', 'Economicus.perseus-grc2.xml', 'Hellenica.perseus-grc2.xml', 'Herodotus.xml', 'Hiero.perseus-grc2.xml', 'Memorabilia.perseus-grc2.xml', 'Symposium.perseus-grc2.xml', 'Thucydides.xml', 'tlg0010.tlg001.perseus-grc2.xml', 'tlg0010.tlg002.perseus-grc2.xml', 'tlg0010.tlg003.perseus-grc2.xml', 'tlg0010.tlg004.perseus-grc2.xml', 'tlg0010.tlg005.perseus-grc2.xml', 'tlg0010.tlg006.perseus-grc2.xml', 'tlg0010.tlg007.perseus-grc2.xml', 'tlg0010.tlg008.perseus-grc2.xml', 'tlg0010.tlg009.perseus-grc2.xml', 'tlg0010.tlg010.perseus-grc2.xml', 'tlg0010.tlg011.perseus-grc2.xml', 'tlg0010.tlg012.perseus-grc2.xml', 'tlg0010.tlg013.perseus-grc2.xml', 'tlg0010.tlg014.perseus-grc2.xml', 'tlg0010.tlg015.perseus-grc2.xml', 'tlg0010.tlg016.perseus-grc2.xml', 'tlg0010.tlg017.perseus-grc2.xml', 'tlg0010.tlg018.perseus-grc2.xml', 'tlg0010.tlg019.perseus-grc2.xml', 'tlg0010.tlg020.perseus-grc2.xml', 'tlg0010.tlg021.perseus-grc2.xml', 'tlg0010.tlg022.perseus

In [12]:
for work in listOfFiles:

    file = open(os.path.join(path, work), "r", encoding="utf-8")
    content = file.read()
    file.close()
    content = re.sub(r'<\?[^>]*>', '', content)
    content = re.sub(r'<TEI[^>]*>', '', content)
    content = re.sub(r'[\t|\r]', '', content)
    content = re.sub(r'[\s]+', ' ', content)
    content = re.sub(r'\n', '', content)
    content = re.sub(r'<teiHeader>.*<body>', '', content)
    content = re.sub(r'>', '>\n', content)

    #print(content)
    
    #Strip out the extra tags (milestone, persname, q, placename, etc)

    #matches tags, but not their contents
    cleantext = re.sub(r'</?persName>', '', content)
    cleaner = re.sub(r'</?placeName[^<]*>', '', cleantext)
    cleaner = re.sub(r'<milestone[^/]*/>', '', cleaner)
    cleaner = re.sub(r'</?foreign[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?quote[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?gloss[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?title[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?date[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?surname[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?sic[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?corr[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?choice[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?said[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?q[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?body[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?text[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?TEI[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?del[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?add[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?emph[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?l[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?cit[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?term[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?sp[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?gap[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?hi[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?anchor[^>]*>', '', cleaner)
    cleaner = re.sub(r'</?num[^>]*>', '', cleaner)

    tags = []
    allowed = ['</p>', '</div>', '</note>', '</head>', '</bibl>']
    tags = re.findall(r'</[^[>|/]*/?>', cleaner)
    unique_tags = [] 
    [unique_tags.append(x) for x in tags if x not in unique_tags] 
    #print(unique_tags)
    bad_tags = []
    [bad_tags.append(x) for x in unique_tags if x not in allowed] 
    print(bad_tags)
    
    #Get rid of note tags and their content

    cleaner = re.sub(r'<note[^>]*>[^<]*</note>', '', cleaner)
    cleaner = re.sub(r'<head[^>]*>[^<]*</head>', '', cleaner)
    cleaner = re.sub(r'<speaker[^>]*>[^<]*</speaker>', '', cleaner)
    cleaner = re.sub(r'<bibl[^>]*>[^<]*</bibl>', '', cleaner)
    cleaner = re.sub(r'<!--[^>]*-->', '', cleaner)

#get rid of <p> and <div> for now. Will be useful for cross-validation later

    cleaner = re.sub(r'</?p[^>]*>', '', cleaner)
    #cleaner = re.sub(r'<div[^>]*>', '', cleaner)
    cleaner = re.sub(r'</div>', r'@~', cleaner)

#Get rid of extra whitespace
    cleaner = re.sub(r'\n', ' ', cleaner)
    cleaner = re.sub(r'[\s]+', ' ', cleaner)

#Replace nested </div> tags
    cleaner = re.sub(r'(@~ )+', r'@~ ', cleaner)

#Replace temporary symbols with tab and return characters
    cleaner = re.sub(r'@~', '\n', cleaner)

    #print(cleaner)

    sections = cleaner.split('\n')
    #print(sections)
    work = ''
    book = 0
    chapter = 0
    section = 0
    citation_list = []

    for text in sections: 
        text.strip()
        if text.count('type=\"edition\"') >= 1:
            found = re.search(r'urn[^\"]*', text)
            if found:
                work = found.group(0)
            
            book = 0
            chapter = 0
            section = 0
        
        if text.count('subtype=\"book\"') >= 1:
            book = book + 1
            chapter = 0
            section = 0
        
        if text.count('subtype=\"chapter\"') >= 1:
            chapter = chapter + 1
            section = 0
        
        if text.count('subtype=\"section\"') >= 1:
            section = section + 1
        
        citation = work+'@'+str(book)+"."+str(chapter)+"."+str(section)
        citation_list.append([citation, text])
    
    for item in citation_list:
        item[1] = re.sub(r'<div[^>]*>', '', item[1])
        item[1] = item[1].strip()
    
    #Put the clean text in a file
    cleanfile = open('processed_text/citation_list.tsv', 'a', encoding="utf-8") 
    for item in citation_list:
        cleanfile.write(item[0]+ "\t"+item[1]+"\n")
        
    cleanfile.close()
    
    
    
#This should spit out a bunch of empty lists ("[]"), one for each file. If there is anything in the lists, that 
#means there are tags that are not being removed properly. Figure out which file is the problem (by counting) and
#look for the problem tags in it to see how they need to be removed. If they wrap around stuff that isn't original
#text, they need to be deleted with their contents, like <note>, <head>, or <speaker> tags. Otherwise, they can
#join the massive list above that. 

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
